In [ ]:
"""
Purpose: Run the Soma Finding
Algorithm for all the cells
in our final match



"""


In [13]:
current_version = 30

In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

INFO - 2021-01-13 07:16:57,306 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-13 07:16:57,308 - settings - Setting database.user to celiib
INFO - 2021-01-13 07:16:57,309 - settings - Setting database.password to newceliipass
INFO - 2021-01-13 07:16:57,311 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-01-13 07:16:57,312 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-13 07:16:57,324 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-01-13 07:16:57,590 - settings - Setting enable_python_native_blobs to True


In [2]:
test_mode = False

In [22]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()
nucleus_table = du.configure_nucleus_table(current_version)

INFO - 2021-01-13 07:18:03,682 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-13 07:18:03,684 - settings - Setting database.user to celiib
INFO - 2021-01-13 07:18:03,685 - settings - Setting database.password to newceliipass
INFO - 2021-01-13 07:18:03,690 - settings - Setting enable_python_native_blobs to True


Sleeping 4 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2021-01-13 07:18:03,980 - settings - Setting enable_python_native_blobs to True


# Getting the list of neurons to decompose for the mutli soma testing

In [4]:
# import pandas as pd
# soma_soma_table = pd.read_csv("Minnie65 core proofreading - Soma-Soma.csv")
# no_header = soma_soma_table.iloc[1:]
# multi_soma_ids_str = no_header["Dendrites"].to_numpy()
# multi_soma_ids = multi_soma_ids_str[~np.isnan(multi_soma_ids_str.astype("float"))].astype("int")

# @schema
# class MultiSomaProofread(dj.Manual):
#     definition="""
#     segment_id : bigint unsigned  #segment id for those to be decimated
#     """
    
# dict_of_seg = [dict(segment_id=k) for k in multi_soma_ids]
# minnie.MultiSomaProofread.insert(dict_of_seg,skip_duplicates=True)
# MultiSomaProofread()

# Defining the Table

In [15]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [16]:
import meshlab
meshlab.set_meshlab_port(current_port=None)
temporary_folder = 'decimation_temp'
meshlab_scripts = {}

No port chosen so picked random port 8810


In [17]:
#so that it will have the adapter defined
from datajoint_utils import *

In [18]:
@schema
class NeuronGliaNuclei(dj.Manual):
    definition="""
    -> minnie.Decimation.proj(decimation_version='version')
    ver : decimal(6,2) #the version number of the materializaiton
    ---
    n_glia_faces              : int unsigned                 # The number of faces that were saved off as belonging to glia
    glia_faces=NULL           : <faces>                      # faces indices that were saved off as belonging to glia (external storage)
    n_nuclei_faces            : int unsigned                 # The number of faces that were saved off as belonging to nuclie
    nuclei_faces=NULL         : <faces>                      # faces indices that were saved off as belonging to nuclei (external storage)
    """

In [19]:
# schema.external['faces'].delete(delete_external_files=True)
# schema.external['somas'].delete(delete_external_files=True)

In [20]:
# minnie.BaylorSegmentCentroid.delete()
# minnie.NeuronGliaNuclei().delete()

In [24]:
# decimation_version = 0
# decimation_ratio = 0.25
# verts_min = 10000


# key_source =  ((minnie.Decimation & f"n_vertices > {verts_min}").proj(decimation_version='version') & 
#                         "decimation_version=" + str(decimation_version) &
#                    f"decimation_ratio={decimation_ratio}") & (dj.U("segment_id") & (minnie.OldBaylorSegmentCentroid() & "multiplicity<3").proj()
#                                                              & (dj.U("segment_id") & nucleus_table))
# key_source

In [25]:
decimation_version = 0
decimation_ratio = 0.25
verts_min = 10000
current_version = 30


import trimesh_utils as tu
import soma_extraction_utils as sm
@schema
class BaylorSegmentCentroid(dj.Computed):
    definition="""
    -> minnie.Decimation.proj(decimation_version='version')
    soma_index : tinyint unsigned #index given to this soma to account for multiple somas in one base semgnet
    ver : decimal(6,2) #the version number of the materializaiton
    ---
    centroid_x=NULL           : int unsigned                 # (EM voxels)
    centroid_y=NULL           : int unsigned                 # (EM voxels)
    centroid_z=NULL           : int unsigned                 # (EM voxels)
    n_vertices=NULL           : bigint                 #number of vertices
    n_faces=NULL            : bigint                  #number of faces
    mesh: <somas>  #datajoint adapter to get the somas mesh objects
    multiplicity=NULL         : tinyint unsigned             # the number of somas found for this base segment
    sdf=NULL                  : double                       # sdf width value for the soma
    volume=NULL               : double                       # the volume in billions (10*9 nm^3) of the convex hull
    max_side_ratio=NULL       : double                       # the maximum of the side length ratios used for check if soma
    bbox_volume_ratio=NULL    : double                       # ratio of bbox (axis aligned) volume to mesh volume to use for check if soma
    max_hole_length=NULL      : double                    #euclidean distance of the maximum hole size
    run_time=NULL : double                   # the amount of time to run (seconds)

    """

    key_source =  ((minnie.Decimation & f"n_vertices > {verts_min}").proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}") & (dj.U("segment_id") & (minnie.OldBaylorSegmentCentroid() & "multiplicity<3").proj()
                                                                 & (dj.U("segment_id") & nucleus_table))
     

    def make(self,key):
        """
        Pseudocode: 
        1) Compute all of the
        2) Save the mesh as an h5 py file
        3) Store the saved path as the decomposition part of the dictionary and erase the vertices and faces
        4) Insert
        
        
        """
        
        #get the mesh data
        print(f"\n\n\n---- Working on Neuron {key['segment_id']} ----")
        print(key)
        new_mesh = (minnie.Decimation() & key).fetch1("mesh")
        current_mesh_verts,current_mesh_faces = new_mesh.vertices,new_mesh.faces

        segment_id = key["segment_id"]

        (total_soma_list, 
         run_time, 
         total_soma_list_sdf,
         glia_pieces,
         nuclei_pieces) = sm.extract_soma_center(
                            segment_id,
                            current_mesh_verts,
                            current_mesh_faces,
            return_glia_nuclei_pieces=True,
        )
        
        # -------- 1/9 Addition: Going to save off the glia and nuclei pieces ----------- #
        """
        Psuedocode:
        For both glia and nuclie pieces
        1) If the length of array is greater than 0 --> combine the mesh and map the indices to original mesh
        2) If not then just put None     
        """
        orig_mesh = trimesh.Trimesh(vertices=current_mesh_verts,
                                   faces=current_mesh_faces)
        
        if len(glia_pieces)>0:
            glia_faces = tu.original_mesh_faces_map(orig_mesh,tu.combine_meshes(glia_pieces))
            n_glia_faces = len(glia_faces)
        else:
            glia_faces = None
            n_glia_faces = 0
            
        if len(nuclei_pieces)>0:
            nuclei_faces = tu.original_mesh_faces_map(orig_mesh,tu.combine_meshes(nuclei_pieces))
            n_nuclei_faces = len(nuclei_faces)
        else:
            nuclei_faces = None
            n_nuclei_faces = 0
            
        # --------- saving the nuclei and glia saves
        glia_path,nuclei_path = du.save_glia_nuclei_files(glia_faces=glia_faces,
                                 nuclei_faces=nuclei_faces,
                                 segment_id=segment_id)
        
        print(f" glia_path = {glia_path} \n nuclei_path = {nuclei_path}")
            
        glia_nuclei_key = dict(key,
                               ver=current_version,
                               n_glia_faces=n_glia_faces,
                               #glia_faces = glia_faces,
                               glia_faces = glia_path,
                               n_nuclei_faces = n_nuclei_faces,
                               #nuclei_faces = nuclei_faces
                               nuclei_faces = nuclei_path,
                              )
        
        NeuronGliaNuclei.insert1(glia_nuclei_key,replace=True)
        print(f"Finished saving off glia and nuclei information : {glia_nuclei_key}")
        
        # ---------------- End of 1/9 Addition --------------------------------- #
        
        
        
        print(f"Run time was {run_time} \n    total_soma_list = {total_soma_list}"
             f"\n    with sdf values = {total_soma_list_sdf}")
        
        #check if soma list is empty and did not find soma
        if len(total_soma_list) <= 0:
            print("There were no somas found for this mesh so just writing empty data")
            

            returned_file_path = tu.write_h5_file(
                                                vertices=np.array([]),
                                                  faces=np.array([]),
                                                  segment_id=segment_id,
                                                  filename = f'{segment_id}_0.h5',
                                                    filepath=str(du.get_somas_path())
                                                 )

            
            
            insert_dict = dict(key,
                              soma_index=0,
                               ver=current_version,
                              centroid_x=None,
                               centroid_y=None,
                               centroid_z=None,
                               #distance_from_prediction=None,
                               #prediction_matching_index = None,
                               n_vertices=0,
                               n_faces=0,
                               mesh=returned_file_path,
                               multiplicity=0,
                               sdf = None,
                               volume = None,
                               max_side_ratio = None,
                               bbox_volume_ratio = None,
                               max_hole_length=None,
                               run_time=run_time
                              )
            
            #raise Exception("to prevent writing because none were found")
            self.insert1(insert_dict,skip_duplicates=True)
            return
        
        #if there is one or more soma found, get the volume and side length checks
        max_side_ratio =  [np.max(sm.side_length_ratios(m)) for m in total_soma_list]
        bbox_volume_ratio =  [sm.soma_volume_ratio(m) for m in total_soma_list]
        dicts_to_insert = []


        for i,(current_soma,soma_sdf,sz_ratio,vol_ratio) in enumerate(zip(total_soma_list,total_soma_list_sdf,max_side_ratio,bbox_volume_ratio)):
            print("Trying to write off file")
            """ Currently don't need to export the meshes
            current_soma.export(f"{key['segment_id']}/{key['segment_id']}_soma_{i}.off")
            """
            auto_prediction_center = np.mean(current_soma.vertices,axis=0) / np.array([4,4,40])
            auto_prediction_center = auto_prediction_center.astype("int")
            print(f"Predicted Coordinates are {auto_prediction_center}")
            max_hole_length = tu.largest_hole_length(current_soma)
            
            returned_file_path = tu.write_h5_file(
                                            vertices=current_soma.vertices,
                                              faces=current_soma.faces,
                                              segment_id=segment_id,
                                              filename = f'{segment_id}_{i}.h5',
                                                filepath=str(du.get_somas_path())
                                             )



            insert_dict = dict(key,
                              soma_index=i+1,
                               ver=current_version,
                              centroid_x=auto_prediction_center[0],
                               centroid_y=auto_prediction_center[1],
                               centroid_z=auto_prediction_center[2],
                               n_vertices = len(current_soma.vertices),
                               n_faces = len(current_soma.faces),
                               mesh=returned_file_path,
                               multiplicity=len(total_soma_list),
                               sdf = np.round(soma_sdf,3),
                               volume = current_soma.convex_hull.volume/1000000000,
                               max_side_ratio = np.round(sz_ratio,3),
                               bbox_volume_ratio = np.round(vol_ratio,3),
                               max_hole_length = np.round(max_hole_length,3),
                               run_time=np.round(run_time,4)
                              )



            dicts_to_insert.append(insert_dict)
        self.insert(dicts_to_insert,skip_duplicates=True)

# Running the Populate

In [26]:
curr_table = (minnie.schema.jobs & "table_name='__baylor_segment_centroid'")
curr_table#.delete()# & "status='error'"#.delete()
#curr_table.delete()

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__baylor_segment_centroid,006d4a754852db78637a9730066c3558,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.147,at-node20,29,34606,2021-01-13 01:03:52
__baylor_segment_centroid,0de9544238e2741166282bce76c5b67b,reserved,=BLOB=,,=BLOB=,celiib@10.66.4.26,at-compute006,29,34582,2021-01-13 01:05:15
__baylor_segment_centroid,0e0a23b50c00948d31c0e936b968e570,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.142,at-node15,29,34616,2021-01-13 01:03:46
__baylor_segment_centroid,1071bede410fb82537727d706a6779da,reserved,=BLOB=,,=BLOB=,celiib@10.66.4.26,at-compute006,29,34583,2021-01-13 01:16:47
__baylor_segment_centroid,12dc98a8b9a21eadf99f39bdbf6efd06,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.164,at-node37,29,34631,2021-01-13 00:59:13
__baylor_segment_centroid,159fb5a371bd94dd452794c391f31487,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.165,at-node38,29,34579,2021-01-13 01:15:14
__baylor_segment_centroid,181b67c7b236e63d1bb947f25fb21690,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.132,at-node5,29,34558,2021-01-13 01:07:54
__baylor_segment_centroid,1bb2b246f4acc8687ff62021801e8c0f,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.172,at-node45,29,34557,2021-01-13 01:11:04
__baylor_segment_centroid,1dd6397125b48352b3a53e23b5447f81,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.135,at-node8,30,34563,2021-01-13 01:18:09
__baylor_segment_centroid,223292d754d01c10ec97cae17b086998,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.144,at-node17,29,34601,2021-01-13 01:02:05


In [ ]:
# import pandas as pd
# key_hash,error_message = curr_table.fetch("key_hash","error_message")

# df = pd.DataFrame.from_dict([dict(key_hash = k,error_message = m) for k,m in zip(key_hash,error_message)])
# df
# #df.columns = ["error","key_hash"]
# key_hashes_to_delete = df[df["error_message"].str.contains("OSError")]["key_hash"].to_numpy()

# (curr_table & [dict(key_hash=k) for k in key_hashes_to_delete]).delete()

In [ ]:
import time
import random

sm = reload(sm)

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    BaylorSegmentCentroid.populate(reserve_jobs=True, suppress_errors=True)
else:
    BaylorSegmentCentroid.populate(reserve_jobs=True, suppress_errors=False)
print('Populate Done')

print(f"Total time for BaylorSegmentCentroid populate = {time.time() - start_time}")

Populate Started


INFO - 2021-01-13 07:20:47,404 - autopopulate - Found 75752 keys to populate
INFO - 2021-01-13 07:20:47,429 - connection - Transaction started
INFO - 2021-01-13 07:20:47,431 - autopopulate - Populating: {'segment_id': 864691136050815731, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25')}





---- Working on Neuron 864691136050815731 ----
{'segment_id': 864691136050815731, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25')}
Current Arguments Using (adjusted for decimation):
 large_mesh_threshold= 5000.0 
large_mesh_threshold_inner = 3250.0 
soma_size_threshold = 562.5 
soma_size_threshold_max = 75000.0
outer_decimation_ratio = 0.25
inner_decimation_ratio = 0.25
max_mesh_sized_filtered_away = 22500.0
xvfb-run -n 8810 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Auto_Proofreading/Preprocessing_Pipeline/temp/neuron_49109.off -o /notebooks/Auto_Proofreading/Preprocessing_Pipeline/temp/neuron_49109_remove_interior.off -s /notebooks/Auto_Proofreading/Preprocessing_Pipeline/temp/remove_interior_800606.mls
removed temporary input file: /notebooks/Auto_Proofreading/Preprocessing_Pipeline/temp/neuron_49109.off
removed temporary output file: /notebooks/Auto_Proofreading/Preprocessing_Pipeline/temp/neuron_49109_remove_interior.off
/notebooks/Auto_Proofreading/

/meshAfterParty/trimesh_utils.py:662: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  ordered_comp_indices = np.array([k.astype("int") for k in ordered_components])


There were 4 total interior meshes
Pieces satisfying glia requirements (volume) (x >= 2500000000000): 0
Pieces satisfying nuclie requirements: n_faces (700 <= x) and volume (x < 2500000000000) : 4
inside remove_mesh_interior and using precomputed inside_pieces
Removing the following inside neurons: [<trimesh.Trimesh(vertices.shape=(53342, 3), faces.shape=(104640, 3))>, <trimesh.Trimesh(vertices.shape=(2493, 3), faces.shape=(3987, 3))>, <trimesh.Trimesh(vertices.shape=(1088, 3), faces.shape=(1749, 3))>, <trimesh.Trimesh(vertices.shape=(722, 3), faces.shape=(1163, 3))>]


Original Mesh size: 190718, Final mesh size: 79179
Total time = 12.517207145690918
xvfb-run -n 8810 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Auto_Proofreading/Preprocessing_Pipeline/864691136050815731/neuron_864691136050815731.off -o /notebooks/Auto_Proofreading/Preprocessing_Pipeline/864691136050815731/neuron_864691136050815731_decimated.off -s /notebooks/Auto_Proofreading/Preprocessing_Pipeline/864691

...

/meshAfterParty/trimesh_utils.py:2835: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  ordered_comp_indices = np.array([k.astype("int") for k in ordered_components])


Total found significant pieces AFTER Poisson = [<trimesh.Trimesh(vertices.shape=(6773, 3), faces.shape=(13542, 3))>]
----- working on mesh after poisson #0: <trimesh.Trimesh(vertices.shape=(6773, 3), faces.shape=(13542, 3))>
xvfb-run -n 8810 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Auto_Proofreading/Preprocessing_Pipeline/864691135758479438/neuron_864691135758479438_decimated_largest_piece_poisson_largest_inner.off -o /notebooks/Auto_Proofreading/Preprocessing_Pipeline/864691135758479438/neuron_864691135758479438_decimated_largest_piece_poisson_largest_inner_decimated.off -s /notebooks/Auto_Proofreading/Preprocessing_Pipeline/864691135758479438/decimation_meshlab_25328866.mls

-------Splits after inner decimation len = 1--------

done exporting decimated mesh: neuron_864691135758479438_decimated_largest_piece_poisson_largest_inner.off

    --- On segmentation loop 0 --
largest_mesh_path_inner_decimated_clean = <trimesh.Trimesh(vertices.shape=(1693, 3), faces.shape=(338


Largest hole before segmentation = 31881.212945100524, after = 109132.35405044594,
ratio = 3.423092911752509, difference = 77251.14110534542
Saved object at /mnt/dj-stor01/platinum/minnie65/02/glia_nuclei_faces/864691135758479438_glia.pbz2
File size is 4.6e-05 MB
Saved object at /mnt/dj-stor01/platinum/minnie65/02/glia_nuclei_faces/864691135758479438_nuclei.pbz2
File size is 0.021275 MB
 glia_path = /mnt/dj-stor01/platinum/minnie65/02/glia_nuclei_faces/864691135758479438_glia.pbz2 
 nuclei_path = /mnt/dj-stor01/platinum/minnie65/02/glia_nuclei_faces/864691135758479438_nuclei.pbz2
Finished saving off glia and nuclei information : {'segment_id': 864691135758479438, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25'), 'ver': 30, 'n_glia_faces': 0, 'glia_faces': PosixPath('/mnt/dj-stor01/platinum/minnie65/02/glia_nuclei_faces/864691135758479438_glia.pbz2'), 'n_nuclei_faces': 18832, 'nuclei_faces': PosixPath('/mnt/dj-stor01/platinum/minnie65/02/glia_nuclei_faces/864691135758479438

/meshAfterParty/trimesh_utils.py:1457: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  total_submeshes_idx =np.array(list(total_submeshes_idx.values()))
/meshAfterParty/networkx_utils.py:556: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  cycles_list_array = np.array(cycles_list)


Using Poisson Surface Reconstruction for watertightness in soma_volume_ratio
xvfb-run -n 8810 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Auto_Proofreading/Preprocessing_Pipeline/Poisson_temp/neuron_791299.off -o /notebooks/Auto_Proofreading/Preprocessing_Pipeline/Poisson_temp/neuron_791299_poisson.off -s /notebooks/Auto_Proofreading/Preprocessing_Pipeline/Poisson_temp/poisson_908936.mls
